# Linear sorting

All sorting utilizing the comparison model are lower bounded by O(nlog(n)). The direct access arrays have O(1) read and write times. Therefore by leveraging direct access arrays sorting can be performed in O(n). 

The rest of notebook goes through the developement of a sorting algorithm (radix sort) that achieves O(n) sorting performance. 

## Direct Access Sort

This data structure requires unique keys. It instantiates a direct access array of length u. Where is the max range of values of the keys can take. The keys are taken from the set [0,u]. 

Fist a direct access array of length u needs to be instantiated costing O(u).

Secondly, the items need to be inserted into the direct access array. Each insert operation run in constant time resulting in a total cost of O(n) to insert all items.

Then the direct access array needs to be iterated through yeilding its stored value if present. This returns a sorted version of the list and runs in O(u).

Cons:
- The issues with direct access array is that it assumes unique keys, otherwise collision exists for the direct access array. 
- Moreover it works well only when u=O(n), otherwise sorting complexity is more than O(n) 

In [1]:
# %load -s direct_access_sort modules/linear_sort.py
def direct_access_sort(A):
    u=1+max([a.key for a in A]) #O(n)
    D=[None]*u #O(u)
    for a in A: #O(n)
        D[a.key]=a
    i=0
    for d in D: #O(u)
        if d is not None:
            A[i]=d
            i+=1


In [2]:
a_key=[2,4,9,3,6,12]
A=[type('Node', (object,), {'key': key}) for key in a_key]
print([a.key for a in A])
direct_access_sort(A)
print([a.key for a in A])

[2, 4, 9, 3, 6, 12]
[2, 3, 4, 6, 9, 12]


## Counting Sort 

Conting sort solves the issue of not being able to sort lists with duplicate keys. To do so at each index in the direct access array another data structure is used to store the keys. This would help in the event of collisions arising from duplicate keys. For radix sort it is important that the order of the duplicate keys is the same for the output from sorting as it is for the input (i.e stable sorting). To do so a sequence data structure needs to be used at each index of the direct access array. In the implementation below a python list is used to do so. 

The rest of the operations involved in the sorting is the same as the direct access array. 

In [3]:
# %load -s counting_sort modules/linear_sort.py
def counting_sort(A):
    u=1+max([a.key for a in A]) #0(n)
    D=[[] for _ in range(u)] #O(u)
    for a in A: #O(n)
        D[a.key].append(a)

    i=0
    for chain in D: #O(u)
        for x in chain:
            A[i]=x
            i+=1


In [4]:
a_key=[2,4,9,3,9,6,12]
A=[type('Node', (object,), {'key': key}) for key in a_key]
print([a.key for a in A])
counting_sort(A)
print([a.key for a in A])

[2, 4, 9, 3, 9, 6, 12]
[2, 3, 4, 6, 9, 9, 12]


## Radix Sort
- As long as logn(u)=constant radix sort can sort items in linear time 
- It can handle repeated keys like counting sort

In [5]:
# %load -s radix_sort modules/linear_sort.py
def radix_sort(A):
    u = 1 + max([a.key for a in A])
    n = len(A)
    c = 1 + u.bit_length()//n.bit_length()

    D=[]
    class Obj: pass

    for i in range(n): #O(cn)
        D.append(Obj())
        D[i].item=A[i]
        D[i].digits=[]
        high=A[i].key
        for j in range(c):
            high,low=divmod(high,n)
            D[i].digits.append(low)
    
    for i in range(c): #O(cn)
        for j in range(n):
            D[j].key=D[j].digits[i]
        counting_sort(D)
    
    for i,d in enumerate(D): #O(n)
        A[i]=d.item 

    return A


In [6]:
a_key=[2,4,3,1,9,14,4,9,3,6,12]
A=[type('Node', (object,), {'key': key}) for key in a_key]
print([a.key for a in A])
radix_sort(A)
print([a.key for a in A])

[2, 4, 3, 1, 9, 14, 4, 9, 3, 6, 12]
[1, 2, 3, 3, 4, 4, 6, 9, 9, 12, 14]
